In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA default;

In [0]:
# --- Konfiguration ---
volume_path = "/Volumes/workspace/default/volume"
source_data_path = f"{volume_path}/enrollments_raw" # Eigener Ordner für die Quelldateien

# Pfade für die Checkpoints jeder Schicht
chk_bronze = f"{volume_path}/_chk/medallion_bronze"
chk_silver = f"{volume_path}/_chk/medallion_silver"
chk_gold = f"{volume_path}/_chk/medallion_gold"

In [0]:
# --- Aufräumen ---
# 1. Tabellen löschen
spark.sql("DROP TABLE IF EXISTS students")
spark.sql("DROP TABLE IF EXISTS enrollments")
spark.sql("DROP TABLE IF EXISTS enrollments_bronze")
spark.sql("DROP TABLE IF EXISTS enrollments_silver")
spark.sql("DROP TABLE IF EXISTS daily_student_courses")
print("✅ Alte Tabellen gelöscht.")

# 2. Checkpoints und Quelldaten-Ordner löschen
dbutils.fs.rm(source_data_path, recurse=True)
dbutils.fs.rm(chk_bronze, recurse=True)
dbutils.fs.rm(chk_silver, recurse=True)
dbutils.fs.rm(chk_gold, recurse=True)
print("✅ Alte Checkpoints und Quelldaten gelöscht.")

✅ Alte Tabellen gelöscht.
✅ Alte Checkpoints und Quelldaten gelöscht.


# Ausgangssituation erstellen

In [0]:
%python
dataset_school = "/Volumes/workspace/default/volume"

all_files = dbutils.fs.ls(dataset_school)
json_files = [f for f in all_files if f.name.endswith(".json")]

display(json_files)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
# JSON lesen:
enrollments_df = spark.read.json("/Volumes/workspace/default/volume/enrollments.json")
enrollments_df.createOrReplaceTempView("enrollments")

In [0]:
# DataFrame als neue JSON-Dateien in den überwachten Ordner schreiben.
# Spark erstellt hier einen Ordner mit part-Dateien, was für den Auto Loader perfekt ist.
(enrollments_df.write
               .mode("overwrite")
               .format("json")
               .save(source_data_path))
               
print(f"✅ Quelldaten wurden in den Auto Loader-Ordner '{source_data_path}' geschrieben.")
print("\n🚀 System ist bereit für den ersten Durchlauf der Medallion-Pipeline.")

✅ Quelldaten wurden in den Auto Loader-Ordner '/Volumes/workspace/default/volume/enrollments_raw' geschrieben.

🚀 System ist bereit für den ersten Durchlauf der Medallion-Pipeline.


In [0]:
# Prüfen ob die Ausgangslage gegeben ist:
# 1. Lese die JSON-Dateien aus dem Ordner in einen neuen DataFrame
check_df = spark.read.format("json").load(source_data_path)

# 2. Zeige den Inhalt des DataFrames direkt mit display() an
print("Inhalt des Quellordners für den Auto Loader:")
display(check_df)

Inhalt des Quellordners für den Auto Loader:


courses,enroll_id,quantity,student_id,timestamp,total
List(101),e001,1,1,2025-08-01T09:00:00,850
"List(102, 103)",e002,2,2,2025-08-02T10:00:00,1800
List(103),e003,1,3,2025-08-03T11:30:00,600


Nun benötigen wir noch Students

In [0]:
import pyspark.sql.functions as F

# --- Statische Lookup-Tabelle für Studenten ---
# Wir lesen die Daten aus der bereitgestellten students.json
students_lookup_df = spark.read.json(f"{volume_path}/students.json")

print("✅ Studenten-Lookup-Tabelle aus 'students.json' erstellt.")
# Optional: Anzeigen, um die geladenen Daten zu prüfen
# display(students_lookup_df) 

✅ Studenten-Lookup-Tabelle aus 'students.json' erstellt.


# Bronze-Schicht – Inkrementelles Laden mit Auto Loader



Diese Zelle liest die JSON-Dateien aus dem Quellordner und schreibt sie in die enrollments_bronze-Tabelle.

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col # Wichtig für den Zugriff auf die _metadata-Spalte

# Bronze-Stream mit Auto Loader
(spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "json")
      .option("cloudFiles.inferColumnTypes", "true")
      .option("cloudFiles.schemaLocation", chk_bronze)
      .load(source_data_path)
      .select("*",
              F.current_timestamp().alias("arrival_time"),
              col("_metadata.file_path").alias("source_file"))
 .writeStream
      .format("delta")
      .option("checkpointLocation", chk_bronze)
      .outputMode("append")
      .trigger(availableNow=True) # Wichtig für Free Edition
      .table("enrollments_bronze")
      .awaitTermination() # Warten, bis dieser Schritt fertig ist
)

print("✅ Bronze-Schicht erfolgreich verarbeitet.")
display(spark.table("enrollments_bronze"))

✅ Bronze-Schicht erfolgreich verarbeitet.


courses,enroll_id,quantity,student_id,timestamp,total,_rescued_data,arrival_time,source_file
List(101),e001,1,1,2025-08-01T09:00:00,850,null,2025-08-28T12:05:38.666Z,/Volumes/workspace/default/volume/enrollments_raw/part-00000-tid-5556120840751717610-d7630ecd-41e2-4f16-82f5-79b1a487a9f8-1146-1-c000.json
"List(102, 103)",e002,2,2,2025-08-02T10:00:00,1800,null,2025-08-28T12:05:38.666Z,/Volumes/workspace/default/volume/enrollments_raw/part-00000-tid-5556120840751717610-d7630ecd-41e2-4f16-82f5-79b1a487a9f8-1146-1-c000.json
List(103),e003,1,3,2025-08-03T11:30:00,600,null,2025-08-28T12:05:38.666Z,/Volumes/workspace/default/volume/enrollments_raw/part-00000-tid-5556120840751717610-d7630ecd-41e2-4f16-82f5-79b1a487a9f8-1146-1-c000.json
List(103),e004,1,1,2025-08-04T11:30:00,600,null,2025-08-28T12:07:20.709Z,/Volumes/workspace/default/volume/enrollments_raw/enrollments_part2.json
List(101),e005,1,2,2025-08-05T11:30:00,850,null,2025-08-28T12:07:20.709Z,/Volumes/workspace/default/volume/enrollments_raw/enrollments_part2.json


# Silber-Schicht – Bereinigen und Anreichern

Diese Zelle liest aus der Bronze-Tabelle, filtert die Daten und reichert sie durch einen Join mit statischen Studentendaten an.

In [0]:
import pyspark.sql.functions as F

# Silber-Stream: Liest von Bronze, bereinigt und reichert an
enrollments_enriched_df = (
    spark.readStream
         .table("enrollments_bronze")
         .where("quantity > 0")
         .withColumn("processed_timestamp", F.to_timestamp("timestamp"))
         .join(students_lookup_df, "student_id")
         .select("enroll_id", "student_id", "email", "gpa", "profile", "quantity", "courses", "processed_timestamp")
)

# Ergebnis in die Silber-Tabelle schreiben
(enrollments_enriched_df.writeStream
                        .format("delta")
                        .option("checkpointLocation", chk_silver)
                        .option("mergeSchema", "true")
                        .outputMode("append")
                        .trigger(availableNow=True)
                        .table("enrollments_silver")
                        .awaitTermination()
)

print("✅ Silber-Schicht erfolgreich verarbeitet.")
display(spark.table("enrollments_silver"))

✅ Silber-Schicht erfolgreich verarbeitet.


enroll_id,student_id,email,gpa,profile,quantity,courses,processed_timestamp
e001,1,anna@example.com,3.9,full-time,1,List(101),2025-08-01T09:00:00.000Z
e002,2,ben@example.com,3.2,part-time,2,"List(102, 103)",2025-08-02T10:00:00.000Z
e003,3,clara@example.com,3.7,exchange,1,List(103),2025-08-03T11:30:00.000Z
e004,1,anna@example.com,3.9,full-time,1,List(103),2025-08-04T11:30:00.000Z
e005,2,ben@example.com,3.2,part-time,1,List(101),2025-08-05T11:30:00.000Z


# Gold-Schicht – Aggregation für Business-Insights

Diese Zelle liest aus der Silber-Tabelle und erstellt eine aggregierte Sicht für das Reporting.

In [0]:
import pyspark.sql.functions as F

# Gold-Stream: Liest von Silber und aggregiert die Daten
enrollments_agg_df = (
    spark.readStream
         .table("enrollments_silver")
         .withColumn("day", F.date_trunc("DD", "processed_timestamp"))
         .groupBy("student_id", "email", "day")
         .agg(F.sum("quantity").alias("total_courses_enrolled"))
)

# Ergebnis in die Gold-Tabelle schreiben
(enrollments_agg_df.writeStream
                   .format("delta")
                   .outputMode("complete") # Wichtig für Aggregationen
                   .option("checkpointLocation", chk_gold)
                   .trigger(availableNow=True)
                   .table("daily_student_courses")
                   .awaitTermination()
)

print("✅ Gold-Schicht erfolgreich verarbeitet.")
display(spark.table("daily_student_courses"))

✅ Gold-Schicht erfolgreich verarbeitet.


student_id,email,day,total_courses_enrolled
3,clara@example.com,2025-08-03T00:00:00.000Z,1
1,anna@example.com,2025-08-04T00:00:00.000Z,1
1,anna@example.com,2025-08-01T00:00:00.000Z,1
2,ben@example.com,2025-08-02T00:00:00.000Z,2
2,ben@example.com,2025-08-05T00:00:00.000Z,1


# Neue Daten simulieren

Jetzt simulieren wir, dass ein neuer Tag anbricht und neue Daten eintreffen.



In [0]:
# Inhalt für eine zweite Datei
json_content_part2 = """
{"enroll_id": "e004", "timestamp": "2025-08-04T11:30:00", "student_id": 1, "quantity": 1, "total": 600, "courses": [103]}
{"enroll_id": "e005", "timestamp": "2025-08-05T11:30:00", "student_id": 2, "quantity": 1, "total": 850, "courses": [101]}
"""

# Die neue Datei in den Quellordner schreiben
dbutils.fs.put(f"{source_data_path}/enrollments_part2.json", json_content_part2, overwrite=True)

print("✅ Neue Datendatei 'enrollments_part2.json' wurde mit korrektem Schema hinzugefügt.")

Wrote 245 bytes.
✅ Neue Datendatei 'enrollments_part2.json' wurde mit korrektem Schema hinzugefügt.


# Pipeline erneut ausführen und inkrementellen Lauf beobachten

Um die neuen Daten durch die Pipeline zu bewegen, führe einfach die Bronze, Silver und Gold-Schicht erneut in dieser Reihenfolge aus. Jede Schicht wird dank der Checkpoints nur die neuen Daten verarbeiten. Sie werden sehen, wie sich die Tabellen am Ende aktualisieren.

# Aufräumen

In [0]:
%sql
DROP TABLE IF EXISTS daily_student_courses;
DROP TABLE IF EXISTS enrollments_bronze;
DROP TABLE IF EXISTS enrollments_silver;